In [2]:
# importar bibliotecas 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from config import Config
from src.data_processing import DataDict, ItemDict
import json
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


from IPython.display import display

In [3]:
# diretório dos dados
data_dirs = Config.get_path_dir_data()
# caminho para o arquivo csv da variáveis e suas categorias
path_dict = data_dirs["DICIONÁRIO"]["MICRODADOS_ENEM_2023.csv"]

# caminho para os microdados do enem 2023
path_microdados = data_dirs["DADOS"]["MICRODADOS_ENEM_2023.csv"]
# caminho para os itens das provas
path_itens = data_dirs["DADOS"]["ITENS_PROVA_2023.csv"]

# Caminho para o dicionário de variveis dos microdados
path_dict_microdados = data_dirs["DICIONÁRIO"]["MICRODADOS_ENEM_2023.csv"]
# Caminho para o dicionário de variveis dos microdados
path_dict_itens = data_dirs["DICIONÁRIO"]["ITENS_PROVA_2023.csv"]

In [4]:
dir_data = data_dirs["DADOS"]
dir_parquet_data = dir_data["MICRODADOS_ENEM_2023_parquet"]
path_parquet_file = dir_data["MICRODADOS_ENEM_2023.parquet"]
path_parquet_itens = dir_data["ITENS_PROVA_2023.parquet"]

In [5]:
list_parquet = list(dir_parquet_data.list_file().values())

## **Dicionário das variáveis**

In [6]:
dict_microdados = DataDict(path_dict_microdados)
dict_itens = ItemDict(path_dict_itens)

## **Carregar dados**

### **Carregar Microdado enem**

In [ ]:
df = pd.read_parquet(path_parquet_file)

In [7]:
df1 = pd.read_parquet(list_parquet[0])
df2 = pd.read_parquet(list_parquet[0])
df = pd.concat([df1,df2])

### **Carregar Itens**

In [8]:
df_items = pd.read_parquet(path_parquet_itens)

# **Perguntas**

### **"O tipo de escola afeta a nota dos alunos em matemática? Isso pode evidenciar a disparidade de notas no ambiente rural e urbano?"**

In [ ]:
# variáveis importantes
var_names_escola_local = ['TP_ESCOLA', 'TP_LOCALIZACAO_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'NU_NOTA_MT']

In [ ]:
# data frame